In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else ''
print(device)
block_size=8
batch_size=4
max_iters = 1000
learning_rate = 3e-4
eval_iters=250


cuda


In [2]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:''.join([int_to_string[i] for i in l]) 

data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
n = int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]
 
def get_batch(split):
    data=train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)                                                                                                                                           

inputs:
tensor([[69, 62, 58, 56, 58,  1, 73, 68],
        [ 1, 61, 54, 75, 62, 67, 60,  1],
        [64,  1, 73, 61, 58,  1, 72, 73],
        [71,  1, 61, 58, 54, 57, 72,  1]], device='cuda:0')
targets:
tensor([[62, 58, 56, 58,  1, 73, 68,  1],
        [61, 54, 75, 62, 67, 60,  1, 54],
        [ 1, 73, 61, 58,  1, 72, 73, 71],
        [ 1, 61, 58, 54, 57, 72,  1, 76]], device='cuda:0')


In [10]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.item()
        out[split]=losses.mean()
    model.train()
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self,index,targets=None):
        logits=self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,index,max_new_tokens):
        #index is (B,T) array of indices of current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits,loss = self.forward(index)
            #focus only on the last time step
            logits=logits[:,-1,:] #becomes (B,C)
            #apply softmax to get the probabilities
            probs=F.softmax(logits,dim=-1) #(B,C)
            #sample from the distribution
            index_next = torch.multinomial(probs,num_samples=1) #(B,1)
            #append sampled index to the running sequence
            index=torch.cat((index,index_next),dim=1)#(B,T+1)
        return index
model = BigramLanguageModel(vocab_size)
m=model.to(device)
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)



FiyX"z&Y-]dGpdVKXk,jpdq&noB5x(- poaSqJOWAhYDYp KA2xKKn_I?fACohCTo:7,CtznC2(Kt?:Ij.U;:jngygy5Zo;d4I[ IEL_"EMrn6MSiWcjb!MMxUulEi
dLR7GSOZc6?]::&*b!0m4]6fffVR73sIqqEM0;;*J;YJp_N- 2]wMFHLBJQjvz33
A'P3xJWDl]k.C4QM9w﻿SZ-N3V﻿3XGU
MqUiBCtRtEs_
2hCqx;tAN5j-HujR,sd4WJhmpD﻿RoF0'
_olyDFjfcOg_JveqS-Dux w9?UJ*IzpsdG'0p5Xo7PuEikyaAh
V_y1Hu&0(a)eLP;, XWo9yG]ze-fxvwU
:)ehWU.c?Wuq2Z&JD﻿kwZWuMOgS -GN'biJt]70g)v-m:V4iAPVD?CftCPBH&4mo!Ma;*gU.uYDX"iqXh:])9rpwWY95GpE1Tpm&gRLr9Y?RuVW"6.3R2fC1Z
*:)l4mAFRC!wzE
eoJT',lja


In [ ]:
#Create a Pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
for iter in range(max_iters):
    if iter%eval_iters==0:
        print(f'step: {iter}, loss: {losses}')

    #sample a batch of data
    xb,yb=get_batch('train')

    #evaluate the loss
    logits,loss =  model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

4.469812393188477
